# Prompting using DeepSeek API

---

S.Yu. Papulin (papulin.study@yandex.ru)

### Contents

- [Setting up client](#Setting-up-client)
- [Text generation prompting](#Text-generation-prompting)
    - [Responses Api](#Responses-Api)
    - [Chat Completions Api](#Chat-Completions-Api)
- [Zero-shot learning](#Zero-shot-learning)
- [Few-shot learning](#Few-shot-learning)
- [Function calling](#Function-calling)
- [Integration with `llamaIndex`](#Integration-with-llamaIndex)
- [Sources](#Sources)

In [ ]:
# %pip install openai==1.93.1

⚠️ **Warning.** `DeepSeek API` does not yet support image recognition and generation, as well as file uploading

In [ ]:
import os
import json
from pydantic import BaseModel

from dotenv import load_dotenv
from openai import OpenAI

## Setting up client

In [ ]:
def setup_client():
    load_dotenv()
    return OpenAI(
        # Note: You must have your own API key
        api_key=os.getenv("DEEPSEEK_API_KEY"), 
        base_url="https://api.deepseek.com"
    )

In [ ]:
client = setup_client()
client.__dict__

## Text generation prompting

### Responses Api

In [ ]:
# Note: DeepSeek doesn't support
# response = client.responses.create(
#     model="deepseek-chat",
#     instructions="You are a helpful assistant.",
#     input="Provide a list of emergency contacts in Russia?"
# )

### Chat Completions Api

**Text generation**

In [ ]:
SYSTEM_PROMPT = "You are a helpful assistant."

USER_PROMPT = """
Provide a list of emergency contacts in Russia?
"""

In [ ]:
def send_messages(system_prompt, user_prompt, response_format=None):
    return client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        response_format=response_format,
        stream=False
    )

In [ ]:
response = send_messages(
    system_prompt=SYSTEM_PROMPT,
    user_prompt=USER_PROMPT
)
response.id

In [ ]:
response

In [ ]:
list(filter(lambda x: not x.startswith('_'), dir(response)))

In [ ]:
# response.__dict__

In [ ]:
response.choices[0].message

In [ ]:
print(response.choices[0].message.content)

**Formatted output**

In [ ]:
USER_PROMPT = """
Provide a list of emergency contacts in Russia? Output the list in json format 
without any additional information. Follow the example below:

[{
    "name": "Fire Service",
    "phone": "101",
    "country": "Russia"
}, ...]
"""

In [ ]:
response = send_messages(
    system_prompt=SYSTEM_PROMPT,
    user_prompt=USER_PROMPT
)
response.id

In [ ]:
print(response.choices[0].message.content)

In [ ]:
content_json = (
    response.choices[0].message.content
    .removeprefix("```json")
    .removesuffix("```")
)
contacts = json.loads(content_json)
contacts

**Json format**

In [ ]:
response = send_messages(
    system_prompt=SYSTEM_PROMPT,
    user_prompt=USER_PROMPT,
    response_format={"type": "json_object"}
)
response.id

In [ ]:
print(response.choices[0].message.content)

In [ ]:
contacts = json.loads(response.choices[0].message.content)["emergency_contacts"]
contacts

**Structured output**

In [ ]:
class EmergencyContact(BaseModel):
    name: str
    phone: str
    country: str

In [ ]:
# Note: DeepSeek doesn't support
# response = client.chat.completions.parse(
#     model="deepseek-chat",
#     messages=[
#         {"role": "system", "content": SYSTEM_PROMPT},
#         {"role": "user", "content": USER_PROMPT},
#     ],
#     response_format=EmergencyContact,
#     # text_format=EmergencyContact
# )

In [ ]:
# Note: DeepSeek doesn't support
# response = client.chat.completions.create(
#     model="deepseek-chat",
#     messages=[
#         {"role": "system", "content": SYSTEM_PROMPT},
#         {"role": "user", "content": USER_PROMPT},
#     ],
#     response_format={
#         "type": "json_schema",
#         "name": "contact_response",
#         "schema": {
#             "type": "object",
#             "properties": {
#                 "emergency_contacts": {
#                     "type": "array",
#                     "description": "list of emergency contacts",
#                     "items": {
#                         "type": "object",
#                         "description": "emergency contact",
#                         "properties": {
#                             "name": {
#                                 "type": "string",
#                                 "description": "name of emergency service",
#                             },
#                             "phone": {
#                                 "type": "string",
#                                 "description": "phone number of emergency service",
#                             },
#                             "country": {
#                                 "type": "string",
#                                 "description": "country of emergency service",
#                             }
#                         },
#                         "additionalProperties": False,
#                         "required": [
#                             "name",
#                             "phone",
#                             "country"
#                         ]
#                     }
#                 },
#                 "additionalProperties": False,
#                 "required": [
#                     "emergency_contacts"
#                 ]
#             }
#         },
#         "strict": True
#     }
# )

In [ ]:
[EmergencyContact(**contact) for contact in contacts]

## Zero-shot learning

In [ ]:
SYSTEM_PROMPT = """

# Identity

You are a detector of quotes of historical figures that labels messages 
as QUOTE or NON-QUOTE.

# Instructions

- Determine the author of a text provided in the user's query.
- Determine whether the whole user's query is exactly QUOTE or NON-QUOTE.
- Your response should only be one of the words "QUOTE" or "NON-QUOTE".
- Add the author of the QUOTE to your response.
- Output in json format without any additional information or commentary.

"""

In [ ]:
USER_PROMPT = "If you're alone, it's a pity."
# USER_PROMPT = "If you're alone, it's a great news."
# USER_PROMPT = "If I have seen further than others, it is by standing upon the shoulders of giants."

In [ ]:
response = send_messages(
    system_prompt=SYSTEM_PROMPT,
    user_prompt=USER_PROMPT
)
print(response.choices[0].message.content)

**Debugging**

In [ ]:
SYSTEM_PROMPT = """

# Identity

You are a detector of quotes of historical figures that labels messages 
as QUOTE or NON-QUOTE.

# Instructions

- Determine the author of a text provided in the user's query.
- Determine whether the whole user's query is exactly QUOTE or NON-QUOTE.
- Give some guesses about authors.
- Give the final QUOTE label if you are absolutely sure.

"""

In [ ]:
response = send_messages(
    system_prompt=SYSTEM_PROMPT,
    user_prompt=USER_PROMPT
)
print(response.choices[0].message.content)

**Final prompt**

In [ ]:
SYSTEM_PROMPT = """

# Identity

You are a detector of quotes of historical figures that labels messages 
as QUOTE or NON-QUOTE.

# Instructions

- Determine the author of a text provided in the user's query.
- Determine whether the whole user's query is exactly QUOTE or NON-QUOTE.
- Give some guesses about authors.
- Your response should only be one of the words "QUOTE" or "NON-QUOTE".
- If it's the QUOTE, add the author to your response.
- Give the final QUOTE label if you are absolutely sure.
- Output in json format without any additional information or commentary.

"""

In [ ]:
response = send_messages(
    system_prompt=SYSTEM_PROMPT,
    user_prompt=USER_PROMPT
)
print(response.choices[0].message.content)

## Few-shot learning

In [ ]:
SYSTEM_PROMPT = """

# Identity

You are a detector of quotes of historical figures that labels messages 
as QUOTE or NON-QUOTE.

# Instructions

- Determine the author of a text provided in the user's query.
- Determine whether the whole user's query is exactly QUOTE or NON-QUOTE.
- Your response should only be one of the words "QUOTE" or "NON-QUOTE".
- Add the author of the QUOTE to your response.
- Output in json format without any additional information or commentary.

# Examples

<user_text id="example-1">
Life’s tragedy is that we get old too soon and wise too late.
</user_text>

<assistant_response id="example-1">
{"label": "QUOTE", "author": "Benjamin Franklin"}
</assistant_response>

<user_text id="example-2">
Your strength is in your breakfast.
</user_text>

<assistant_response id="example-2">
{"label": "NON-QUOTE", "author": "user"}
</assistant_response>

<user_text id="example-3">
Whenever you find yourself on the side of the majority, it is time to pause and reflect.
</user_text>

<assistant_response id="example-3">
{"label": "QUOTE", "author": "Mark Twain"}
</assistant_response>

"""


In [ ]:
USER_PROMPT = "If you're alone, it's a pity."
# USER_PROMPT = "If you're alone, it's a great news!"
# USER_PROMPT = "If I have seen further than others, it is by standing upon the shoulders of giants."

In [ ]:
response = send_messages(
    system_prompt=SYSTEM_PROMPT,
    user_prompt=USER_PROMPT
)
print(response.choices[0].message.content)

## Function calling

**External sources**

In [ ]:
import requests


def get_region_by_ip_address():
    """Get region name by current ip address"""
    # URL for getting your public IP
    IP_SERVICE_URL = "https://api.myip.com"
    # URL for getting your location by your IP
    LOCATION_SERVICE_URL = "http://ip-api.com/json/{}"
    # Request your public IP
    ip__response = requests.get(IP_SERVICE_URL)
    ip__response__data = ip__response.json()
    ip = ip__response__data["ip"]
    # Request your location
    location__response = requests.get(LOCATION_SERVICE_URL.format(ip))
    region_data = location__response.json()
    return f"{region_data['country']}, {region_data['regionName']}"


def get_weather_by_coords(latitude: float, longitude: float) -> str:
    """Get weather by coordinates"""
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m")
    data = response.json()
    return data['current']['temperature_2m']


In [ ]:
# check whether the function works correctly
region_name = get_region_by_ip_address()
region_name

In [ ]:
weather = get_weather_by_coords(55.7487, 37.6187)
weather

**Define functions that will be used as external sources**

In [ ]:
tools = [
    # region name
    {
        "type": "function",
        "function": {
            "name": "get_region_name",
            "description": "Get the user's region name by ip address if it was missed in the query."
            # Note: you can provide the below description to 
            # avoid using system instructions
            # "description": "Call the 'get_region_name' function only if the user's query does not include region information"
        }
    }, 
    # weather
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current temperature for provided coordinates in celsius for specified region.",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {"type": "number"},
                    "longitude": {"type": "number"}
                },
                "required": ["latitude", "longitude"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
]

In [ ]:
# match function name in tools with real function in code
map_tools = {
    "get_region_name": get_region_by_ip_address,
    "get_weather": get_weather_by_coords
}

In [ ]:
def send_messages_with_tools(messages):
    """DeepSeek service"""
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        tools=tools
    )
    return response.choices[0].message

In [ ]:
def call_function(function_name, **kwargs):
    """External services"""
    if function_name in map_tools:
        return map_tools[function_name](**kwargs)
    return None


def send_next_message(message, reset_messages=False):
    if reset_messages:
        messages.clear()
    if hasattr(message, "tool_calls"):
        """Function call needed"""
        tool_call_id = message.tool_calls[0].id
        function_name = message.tool_calls[0].function.name
        function_args = json.loads(message.tool_calls[0].function.arguments)
        external_data = call_function(function_name=function_name, **function_args)
        if external_data:
            messages.append(message)
            messages.append({"role": "tool", "tool_call_id": tool_call_id, "content": str(external_data)})
        else:
            return None
    else:
        """User prompt"""
        messages.append(message)
    return send_messages_with_tools(messages)

**Emergency contacts by ip address**

In [ ]:
# create message flow
messages = [{"role": "user", "content": "What are emergency contacts in my region?"}]

In [ ]:
# run prompt and recieve response that contains tool_calls
response_message = send_messages_with_tools(messages)
response_message

In [ ]:
if response_message.tool_calls:
    print(f"It has function call: {response_message.tool_calls[0].function.name}")

In [ ]:
tool_call = response_message.tool_calls[0]
tool_call_id = tool_call.id
function_name = tool_call.function.name
function_args = json.loads(tool_call.function.arguments)
print("Id:", tool_call_id)
print("Function Name:", tool_call_func_name)
print("Function Args:", function_args)
print(tool_call)

In [ ]:
# fetch my region name from external service by ip address
region_name = call_function(function_name, **function_args)

# add previous response to message flow
messages.append(response_message)
# add my region name to message flow
messages.append({"role": "tool", "tool_call_id": tool_call_id, "content": region_name})

messages

In [ ]:
# provide messages to deepseek chat to complete
# prompt with my region name
response_final_message = send_messages_with_tools(messages)
response_final_message

In [ ]:
print(response_final_message.content)

**Weather by region name**

In [ ]:
# Run the second example to determine the weather of a region
messages = list()
response_message = send_next_message(
    message={"role": "user", "content": "What is the weather in Moscow, Russia?"}
)
response_message

The output is incorrect for us because it suggests calling `get_region_name`. Let's add some instructions.

In [ ]:
SYSTEM_PROMPT = """

# Identity

You are a helpful assistant.

# Instructions

- Extract the region name from the user's query.
- Call the "get_region_name" function only if the user's query does not include region information.

"""

messages = [
    {"role": "system", "content": SYSTEM_PROMPT}
]

In [ ]:
# Run again with instructions
response_message = send_next_message(
    message={"role": "user", "content": "What is the weather in Moscow, Russia?"}
)
response_message

In [ ]:
response_message = send_next_message(message=response_message)
response_message

In [ ]:
print(response_message.content)

In [ ]:
# check the first query with instructions above
response_message = send_next_message(
    message={"role": "user", "content": "What are emergency contacts in my region?"}
)
response_message

In [ ]:
response_message = send_next_message(message=response_message)
response_message

In [ ]:
print(response_message.content)

## Integration with `llamaIndex`

In [ ]:
# %pip install \
# llama-index-core \
# llama-index-llms-deepseek

In [ ]:
from llama_index.llms.deepseek import DeepSeek
from llama_index.core.llms import ChatMessage

In [ ]:
SYSTEM_PROMPT = """
You are a helpful assistant.
"""

In [ ]:
llm = DeepSeek(
    model="deepseek-chat", 
    api_key=os.getenv("DEEPSEEK_API_KEY"),
    system_prompt=SYSTEM_PROMPT
)
llm

**Complete**

In [ ]:
USER_PROMPT = """
Provide a list of emergency contacts in Russia?
"""

In [ ]:
def generate_as_stream(llm, prompt):
    for chunk in llm.stream_complete(prompt):
        print(chunk.delta, end="", flush=True)

In [ ]:
generate_as_stream(llm, USER_PROMPT)

In [ ]:
response = llm.complete(USER_PROMPT)
response

In [ ]:
print(response)

**Chat**

In [ ]:
messages = [
    ChatMessage(role="system", content=SYSTEM_PROMPT),
    ChatMessage(role="user", content="What is the capital of France?"),
]
response = llm.chat(messages)
print(response)

In [ ]:
messages.append(ChatMessage(role="assistant", content=response.message.content))
messages.append(ChatMessage(role="user", content="Russia?"))

In [ ]:
response = llm.chat(messages)
print(response)

**Function calling**

In [ ]:
from llama_index.core.tools import FunctionTool
from llama_index.core.agent.workflow import ReActAgent

In [ ]:
weather_tool = FunctionTool.from_defaults(
    fn=get_weather_by_coords,
    name="get_weather",
    description="Get current temperature for provided coordinates in celsius for specified region.",
)

In [ ]:
agent = ReActAgent(
    tools=[weather_tool], 
    llm=llm
)

In [ ]:
response = await agent.run("What is the weather in Paris?")
response

In [ ]:
response.tool_calls

In [ ]:
print(response)

## Sources

- [OpenAI. Text generation and prompting](https://platform.openai.com/docs/guides/text?api-mode=responses)
- [OpenAI. Function calling](https://platform.openai.com/docs/guides/function-calling?api-mode=responses)